# TERM 3 (1989-1994)

### IMPORTING LIBRARIES 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

### IMPORTING INITIAL DATASET

In [2]:
#importing the json code 
file_name ="3_term.json"
with open(file_name, 'r', encoding='utf-8') as f:
    my_data = json.load(f)

In [3]:
#transforming the json file in a dataframe 
df = pd.json_normalize(my_data["meps"]["mep"])

### ADDING INFORMATION THROUGH SCRAPING 

In [5]:
list_id = df["id"].tolist()
list_name = df["fullName"].tolist()

l=[]
for x in list_name:
    x = x.replace(" ","")
    l.append(x)

In [8]:
#Scraping information about the country for each member 
country_id=[]
for n, code in enumerate(list_id):
    url = f"https://www.europarl.europa.eu/meps/en/{code}/{l[n]}/history/3#detailedcardmep"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    node=soup.find(class_= "erpl_title-h3 mt-1 mb-1")
    if node != None:
        country_id.append(node.text.strip())
    else:
        country_id.append("None")

In [10]:
#creating a column with the countries 
df["country"]= country_id

In [11]:
#Scraping information about eu political party and national party for each member 
main = []
for n, code in enumerate(list_id):
    url = f"https://www.europarl.europa.eu/meps/en/{code}/{l[n]}/history/3#detailedcardmep"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    node=soup.find(class_= "erpl_meps-status-list")
    if node != None:
        main.append(node.text.strip().split("\n\n\n"))
    else:
        main.append("None")

In [12]:
#Add these information to a column in the df
df["text"]= main

### GET NEEDED INFO FROM THE TEXT COLUMN 

In [14]:
#Separating the eu political parties information and the national party information from the text column and removing useless info
first = []
second=[]
for x in range(0,606):
    if df["text"][x] != "None":
        lst = df["text"][x][1].split("\n")
        if len(lst) >1:
            first.append(lst[0])
            second.append(lst[1])
        else:
            first.append(lst[0])
            second.append("None")
    else:
        first.append("NaN")
        second.append("NaN")

In [16]:
#from the list cointaning EU political info, separating the date from the political party name 
eu_group_1 = []
mandate_eu_1=[]
for x in range(0,606):
    lst = first[x].split(":")
    mandate_eu_1.append(lst[0])
    eu_group_1.append(lst[1])

In [17]:
#Creating and adding two separate columns
df["mandate_eu_1"] = mandate_eu_1 #column with 
df["eu_group_1"] = eu_group_1 #column with EU political party affiliation

In [18]:
#Some members have changed political party affiliation during the term, this information is taken from list through a split
eu_group_2 = []
mandate_eu_2=[]
for x in range(0,606):
    if second[x] != "None":
        lst = second[x].split(":")
        mandate_eu_2.append(lst[0])
        eu_group_2.append(lst[1])
    else:
        mandate_eu_2.append("None")
        eu_group_2.append("None")

In [19]:
#creating two new columns referring to the second political party affiliation (if there has been)
df["mandate_eu_2"] = mandate_eu_2
df["eu_group_2"] = eu_group_2

In [21]:
#taking the national party affiliation from the column text
nation_group = []
mandate_nation=[]
for x in range(0,606):
  lst = df["text"][x][3].split(":")
  mandate_nation.append(lst[0])
  nation_group.append(lst[1])

In [22]:
#creating two columns regarding the national party affiliations
df["mandate_nation"] = mandate_nation
df["nation_group"] = nation_group

### CLEANING THE DATASET

In [23]:
#cleaning european political party group columns 
eu_group_1=[]
for x in range(0,606):
    lst = df["eu_group_1"][x].strip().split("-")
    eu_group_1.append(lst[0])
    
eu_group_2=[]
for x in range(0,606):
    if df["eu_group_2"][x] != "None":
        lst = df["eu_group_2"][x].strip().split("-")
        eu_group_2.append(lst[0])
    else:
        eu_group_2.append("None")

df["eu_group_1"]= eu_group_1
df["eu_group_2"]= eu_group_2

In [25]:
#cleaning national political party column

nation_group=[]
for x in range(0,606):
    lst = df["nation_group"][x].strip().split("(")
    nation_group.append(lst[0])
    
for x in nation_group:
    x.strip()

df["nation_group"] = nation_group

In [6]:
#modyfing the string Non with the correct label
df = df.replace("Non", "Non-attached Members")

In [27]:
#cleaning country column

country1=[]
for x in range(0,606):
    lst = df["country"][x].replace("\n","").replace("\t","").replace("\r","").split("-")
    country1.append(lst[0].strip())
df["country"]= country1

In [28]:
#adding the column indicating the term
df["term"]= "1989-1994"

Some members have left before the end of the term and other member have entered in a second moment to substitute them

In [5]:
#Creating a new column which indicates the month and year of the official start as a member of the European Parliament 

df1 = [] 
df1 = df["mandate_eu_1"].tolist()

start_period= []
start_period_1 =[]
end_period_group = []
for x in df1:
    x = x.split("/")
    start_period.append((x[0]).strip())
    end_period_group.append(x[1].strip())
    
for x in start_period:
    x = x.split("-",1)[1]
    start_period_1.append(x)

df["start_period_1"] = start_period_1

In [6]:
#Cleaning the eu_group_2 column 

for x in range(0,606):
    if df["eu_group_2"][x] == df["eu_group_1"][x]:
        df["eu_group_2"][x]="None"
        df["mandate_eu_2"][x] ="None"
    else:
        continue 

C:\Users\gaiaa\AppData\Local\Temp/ipykernel_17440/2240104872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["eu_group_2"][x]="None"
C:\Users\gaiaa\AppData\Local\Temp/ipykernel_17440/2240104872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mandate_eu_2"][x] ="None"


In [7]:
#dropping the columns that are not usefull for the analysis
df = df.drop(["mandate_nation", "mandate_eu_1", "mandate_eu_2","text"], axis=1)

### ADDING THE GENDER COLUMN THROUGH API CALL

In [ ]:
#adding the gender column by using an API that identifies the gender from the name of the person 

def get_gender(name):
    url = f"https://api.genderize.io?name={name}"
    response = requests.get(url).json()
    if response["gender"]:
        return response["gender"]
    else:
        return None

list = []
for element in df.fullName:
    gender = get_gender(element.split()[0])
    list.append(gender)

df["gender"]=list

In [4]:
#creating a new dataframes with the rows that have Nan values in the gender column
df1 = df[df["gender"].isnull()== True].copy()

In [6]:
#adding the correct gender
gender =["female","male","male","male","male","male"]
df1["gender"] = gender

In [7]:
df1

,fullName,id,country,eu_group_1,eu_group_2,nation_group,term,start_period_1,gender
69,Undine-Uta BLOCH von BLOTTNITZ,1815,Germany,Rainbow Group,None,Bündnis 90/die Grünen,1984-1989,07-1984,female
91,Paulin-Christian BRUNÉ,1756,France,Group of the European Democratic Alliance,None,Union pour la démocratie française,1984-1989,04-1986,male
217,Concepció FERRER,1304,Spain,Group of the European People's Party (Christian,None,Convergència i Unió,1984-1989,07-1987,female
253,Carles-Alfred GASÒLIBA i BÖHM,1344,Spain,Liberal and Democratic Reformist Group,None,Convergència i Unió,1984-1989,01-1986,male
272,(Win) Winston James GRIFFITHS,1644,United Kingdom,Socialist Group,None,Labour Party,1984-1989,07-1984,male
276,Julén GUIMON UGARTECHEA,1651,Spain,Group of the European People's Party (Christian,European Democratic Group,Partido Demócrata Popular,1984-1989,01-1986,male
325,(Paddy) Patrick Joseph LALOR,1435,Ireland,Group of the European Democratic Alliance,None,Fianna Fáil Party,1984-1989,07-1984,male
352,Zenon-José LUIS PAZ,1613,Spain,Socialist Group,None,Partido Socialista Obrero Español,1984-1989,01-1986,male
432,(Chris) Christopher Gerard O'MALLEY,1572,Ireland,Group of the European People's Party (Christian,None,Fine Gael Party,1984-1989,06-1986,male
446,(Ben) George Benjamin PATTERSON,1397,United Kingdom,European Democratic Group,None,Conservative and Unionist Party,1984-1989,07-1984,male


In [8]:
#merge the initial dataframe with the dataframe just created
df = df.dropna(axis=0)
frames = [df, df1]
df2 = pd.concat(frames).reset_index()
df2= df2.drop(["index"],axis=1)

### SAVING THE FINAL DATAFRAME

In [10]:
df2.to_csv("term_3_final_gender_full.csv",index=False)